# Package Alma BIBFRAME for export to OCLC (experimental)

The experimental project makes use of an [SRU query](https://na03.alma.exlibrisgroup.com/view/sru/01UPENN_INST?version=1.2&operation=searchRetrieve&recordSchema=lc_bf_instance&query=alma.title_uri=%22sinopia%22&maximum_records=50) to gather all the BF Instances from Sinopia that are found in Alma. In the code cell 3, replace the values for your region and the institution code for your institution.

## Region
``region = 'https://na03.alma.exlibrisgroup.com/'``

## Institution code 
###  replace (01UPENN_INST) with your code
``institution_code = '01UPENN_INST'``

## OCLC Documentation
[OCLC RDF structure](https://help.oclc.org/Metadata_Services/WorldShare_Collection_Manager/Data_sync_collections/Prepare_your_data/Structure_BIBFRAME_data)


In [ ]:
%pip install rdflib
%pip install lxml
%pip install requests
%pip install saxoncee


In [ ]:
# use SRU to get all the BF Instances in Alma

import requests
from lxml import etree

region = 'na03.alma.exlibrisgroup.com'
institution_code = '01UPENN_INST'
url = 'https://'+region+'/view/sru/'+institution_code+'?version=1.2&operation=searchRetrieve&recordSchema=lc_bf_instance&query=alma.title_uri=%22sinopia%22&maximumRecords=50'
response = requests.get(url)
# print(response.text)

# parse the response for the Instance URIs and collect them in a dictionary
root = etree.fromstring(response.content)

instance_dict = {}

# collect all the unique instance URIs and their corresponding bf:sameAs URIs
for record in root.iter('{http://www.loc.gov/zing/srw/}record'):
    record_data = record.find('{http://www.loc.gov/zing/srw/}recordData')
    if record_data is not None:
        rdf = record_data.find('.//{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF')
        if rdf is not None:
            instance = rdf.find('.//{http://id.loc.gov/ontologies/bibframe/}Instance')
            if instance is not None:
                instance_uri = instance.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about')
                if instance_uri:
                    same_as = instance.find('.//{http://id.loc.gov/ontologies/bibframe/}sameAs')
                    if same_as is not None:
                        same_as_uri = same_as.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about')
                        if same_as_uri:
                            instance_dict[instance_uri] = same_as_uri
                        else:
                            print(f"No 'about' attribute found in sameAs element for instance {instance_uri}.")
                    else:
                        print(f"No sameAs element found for instance {instance_uri}.")
                else:
                    print("No 'about' attribute found in Instance element.")
            else:
                print("No Instance element found in RDF element.")
        else:
            print("No RDF element found in recordData element.")
    else:
        print("No recordData element found in record element.")

print("Collected instance URIs and their corresponding sameAs URIs:", instance_dict)
print(f"Total instances collected: {len(instance_dict)}")

# Collect the Instance data

In [3]:
import lxml.etree as ET
from marc_xml.lc_bfxml_work import lc_bfxml_work, remove_last_line
from marc_xml.lc_bfxml_instance import lc_bfxml_instance, remove_rdf_header

In [ ]:
import os
from lxml import etree as ET
from lxml.etree import QName

def rdf2marcxml(instance_dict):
    # Ensure the export directory exists
    export_dir = 'oclc_export'
    os.makedirs(export_dir, exist_ok=True)
    
    for instance_uri, same_as_uri in instance_dict.items():
        # Quality check the file
        
        lc_bfxml_work(instance_uri)  
        remove_last_line() 
        lc_bfxml_instance(instance_uri) 
        remove_rdf_header() 
        
        # Combine the two files, work first
        with open("bfxml_work.xml", "r") as work_file:
            work = work_file.read()
        with open("lc_bfxml_instance.xml", "r") as instance_file:
            instance = instance_file.read()
        
        combined_content = work + instance

        # Save as a file
        combined_file_path = "LoC_Work_Instance.xml"
        with open(combined_file_path, "w") as combined_file:
            combined_file.write(combined_content)

        # Add the sinopiabf and owl namespaces to the combined file
        with open(combined_file_path, "r") as file:
            filedata = file.read()
        filedata = filedata.replace('<rdf:RDF', '<rdf:RDF xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/" xmlns:owl="http://www.w3.org/2002/07/owl#"')
        with open(combined_file_path, "w") as file:
            file.write(filedata)

        # Parse the combined XML
        dom = ET.parse(combined_file_path)
        root = dom.getroot()

        # Define namespaces
        namespaces = {
            'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
            'bf': "http://id.loc.gov/ontologies/bibframe/",
            'owl': "http://www.w3.org/2002/07/owl#"
        }

        # Insert the sameAs link as <owl:SameAs rdf:resource="uri_value"/>
        owl_same_as = ET.Element("{http://www.w3.org/2002/07/owl#}SameAs", attrib={QName(namespaces['rdf'], "resource"): same_as_uri})
        instance_element = root.find('.//bf:Instance[@rdf:about="' + instance_uri + '"]', namespaces=namespaces)
        if instance_element is not None:
            instance_element.append(owl_same_as)
        else:
            print(f"Instance element not found for URI: {instance_uri}")

        # Extract the unique ID from the Sinopia URI
        unique_id = instance_uri.split('/')[-1]

        # Save the updated XML with the unique ID as the file name
        updated_file_path = os.path.join(export_dir, f"{unique_id}.xml")
        with open(updated_file_path, "wb") as f:
            f.write(ET.tostring(root, pretty_print=True, encoding="utf-8"))

        # Apply "pre-transform-normalize.xsl" for normalization
        xslt = ET.parse("marc_xml/xsl/pre-transform-normalize.xsl")
        transform = ET.XSLT(xslt)
        newdom = transform(dom)
        
        normalized_file_path = os.path.join(export_dir, f"{unique_id}_normalized.rdf")
        with open(normalized_file_path, "wb") as f:
            f.write(ET.tostring(newdom, pretty_print=True, encoding="utf-8"))

        print(f"Processed and saved: {normalized_file_path}")

# Expected data model
# instance_dict = {
#     "https://api.stage.sinopia.io/resource/4ac80e0a-d28d-458d-bd7f-5f808023af8d": "https://na03.alma.exlibrisgroup.com/bf/instances/9979461923903681?institute=01UPENN_INST",
#     "https://api.stage.sinopia.io/resource/6dff794b-965b-424f-abcb-cbe16ab9e260": "https://na03.alma.exlibrisgroup.com/bf/instances/9979515924303681?institute=01UPENN_INST"
# }
rdf2marcxml(instance_dict)

# Normalize the data

In [ ]:
import os
from lxml import etree

def apply_final_xslt(instance_dict):
    # Ensure the export directory exists
    export_dir = 'oclc_export'
    os.makedirs(export_dir, exist_ok=True)

    # Load the XSLT file
    xslt_file = 'xsl/rdf2oclc.xsl'
    xslt_doc = etree.parse(xslt_file)
    transform = etree.XSLT(xslt_doc)

    for instance_uri in instance_dict.keys():
        # Extract the unique ID from the Sinopia URI
        unique_id = instance_uri.split('/')[-1]

        # Load the input RDF/XML content
        input_file = os.path.join(export_dir, f"{unique_id}_normalized.rdf")
        with open(input_file, 'r') as f:
            input_xml = f.read()

        # Parse the input XML
        input_doc = etree.fromstring(input_xml)

        # Apply the transformation
        result_doc = transform(input_doc)

        # Save the transformed XML to a file with the unique ID
        output_file = os.path.join(export_dir, f"{unique_id}_final.rdf")
        with open(output_file, 'wb') as f:
            f.write(etree.tostring(result_doc, pretty_print=True, encoding='UTF-8'))

        # Print the transformed XML
        print(f"Transformed XML for {unique_id}:")
        print(etree.tostring(result_doc, pretty_print=True, encoding='UTF-8').decode('UTF-8'))

# Expected data model 
# instance_dict = {
#     "https://api.stage.sinopia.io/resource/4ac80e0a-d28d-458d-bd7f-5f808023af8d": "https://na03.alma.exlibrisgroup.com/bf/instances/9979461923903681?institute=01UPENN_INST",
#     "https://api.stage.sinopia.io/resource/6dff794b-965b-424f-abcb-cbe16ab9e260": "https://na03.alma.exlibrisgroup.com/bf/instances/9979515924303681?institute=01UPENN_INST"
# }
apply_final_xslt(instance_dict)